# Assign spin and remove atoms

In [ ]:
%aiida

In [ ]:
from aiida.orm import StructureData
from aiida.orm import ArrayData
from aiida.engine import calcfunction
from aiida.engine import CalcJob

##### BEGIN add two H
from ase.data import covalent_radii
from ase.neighborlist import NeighborList
import ase.neighborlist
from ase import Atoms
##### END add two H

from base64 import b64decode
from IPython.display import display, clear_output, Image
import ipywidgets as ipw
import nglview
import numpy as np
from numpy.linalg import norm  # add two H

from structure_browser import StructureBrowser

## Step 1: Select a structure from the AiiDA database

In [ ]:
def on_struct_change(c):
    global atoms, removals,addatoms
    node = struct_browser.results.value
    removals = []
    addatoms=Atoms()
    atoms = node.get_ase() if node else None
    if node:
        # ensure that kinds got correctly translated into tags
        for t1, k in zip(atoms.get_tags(), node.get_site_kindnames()):
            t2 = int(k[-1]) if k[-1].isnumeric() else 0
            assert t1==t2
    update_viewer()
    inp_descr.value = node.description + ".edit" if node else ""
    
struct_browser = StructureBrowser()
struct_browser.results.observe(on_struct_change, names='value')
display(struct_browser)

## Step 2: Edit Structure

In [ ]:
def update_viewer():
    global viewer, atoms
    if hasattr(viewer, "component_0"):
        #viewer.clear_representations()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_unitcell()
        cid = viewer.component_0.id
        viewer.remove_component(cid)
    
    if atoms:
        viewer.add_component(nglview.ASEStructure(atoms)) # adds ball+stick
        viewer.add_unitcell()
        viewer.center()
        
        spin_ups = ",".join([str(i) for i, t in enumerate(atoms.get_tags()) if t==1])
        spin_downs = ",".join([str(i) for i, t in enumerate(atoms.get_tags()) if t==2])
        
        viewer.add_representation('ball+stick', selection="@"+spin_ups, color='red', aspectRatio=4.0, opacity=1.0)
        viewer.add_representation('ball+stick', selection="@"+spin_downs, color='green', aspectRatio=4.0, opacity=1.0)

In [ ]:
def on_picked(c):
    global atoms, removals, viewer,  addatoms,positions 
        
    if 'atom1' not in viewer.picked.keys():
        return # did not click on atom
    

    idx = viewer.picked['atom1']['index']
    tags = atoms.get_tags()
    positions=atoms.get_positions()
    atmnums = atoms.get_atomic_numbers()

    node = struct_browser.results.value

    if tool.value == "spin 0":
        tags[idx] = 0
        atoms.set_tags(tags)
    elif tool.value == "spin up":
        tags[idx] = 1
        atoms.set_tags(tags)
    elif tool.value == "spin down":
        tags[idx] = 2
        atoms.set_tags(tags)
    elif tool.value == "kind B":
        atmnums[idx]= 5
        atoms.set_atomic_numbers(atmnums)
    elif tool.value == "kind C":
        atmnums[idx]= 6
        atoms.set_atomic_numbers(atmnums)
    elif tool.value == "kind N":
        atmnums[idx]= 7
        atoms.set_atomic_numbers(atmnums)
    elif tool.value == "remove atom":
        del atoms[idx]
        removals.append(idx)        

    elif tool.value == "z UP":
        dz=0.20 
        positions[idx,2]+= dz
        atoms.set_positions(positions)

    elif tool.value == "z DW":
        dz=0.20 
        positions[idx,2]+= -1.0*dz
        atoms.set_positions(positions)        

    elif tool.value == "add H":
        dCH=1.10 #1.1 for single H
        cov_radii = [covalent_radii[a.number] for a in atoms]
        nl = NeighborList(cov_radii, bothways = True, self_interaction = False)
        nl.update(atoms)
        vec = np.zeros(3)
        indices, offsets = nl.get_neighbors(idx)
        for i, offset in zip(indices, offsets):
            if atoms[i].symbol=='C':
                vec += -atoms[idx].position +(atoms.positions[i] + np.dot(offset, atoms.get_cell()))
        vec = -vec/norm(vec)*dCH
        vec += atoms[idx].position
        vec1 = vec
        first_h  = ase.Atom('H',vec1)
        atoms.append(first_h)
        addatoms.append(first_h)
        atmnums = atoms.get_atomic_numbers()
        tags = atoms.get_tags()

    elif tool.value == "add H2":
        dCH=0.66 #1.1 for single H
        cov_radii = [covalent_radii[a.number] for a in atoms]
        nl = NeighborList(cov_radii, bothways = True, self_interaction = False)
        nl.update(atoms)
        vecz=np.array((0.00,0.00,0.88))
        vec = np.zeros(3)
        indices, offsets = nl.get_neighbors(idx)
        for i, offset in zip(indices, offsets):
            if atoms[i].symbol=='C':
                vec += -atoms[idx].position +(atoms.positions[i] + np.dot(offset, atoms.get_cell()))
        vec = -vec/norm(vec)*dCH
        vec += atoms[idx].position
        vec1 = vec + vecz
        vec2 = vec - vecz
        first_h  = ase.Atom('H',vec1)
        second_h = ase.Atom('H',vec2)
        atoms.append(first_h)
        atoms.append(second_h)
        addatoms.append(first_h)
        addatoms.append(second_h)
        atmnums = atoms.get_atomic_numbers()
        tags = atoms.get_tags()

    elif tool.value == "add H3":
        cov_radii = [covalent_radii[a.number] for a in atoms]
        nl = NeighborList(cov_radii, bothways = True, self_interaction = False)
        nl.update(atoms)

        vec = np.zeros(3)
        indices, offsets = nl.get_neighbors(idx)
        for i, offset in zip(indices, offsets):
            if atoms[i].symbol=='C':
                vec += -atoms[idx].position +(atoms.positions[i] + np.dot(offset, atoms.get_cell()))
        vec = -vec/norm(vec)
        #vec += atoms[idx].position

        vec1 = atoms[idx].position + np.array((0.0,0.35,1.05))
        vec2 = atoms[idx].position + np.array((0.894427,0.35,-0.55))
        vec3 = atoms[idx].position + np.array((-0.894427,0.35,-0.55))
        print(vec,vec1,vec2,vec3)
        first_h  = ase.Atom('H',vec1)
        second_h = ase.Atom('H',vec2)
        third_h  = ase.Atom('H',vec3)
        addatoms.append(first_h)
        addatoms.append(second_h)
        addatoms.append(third_h)
        # Rotate atoms such that y vector is parallel to C-C bond
        addatoms.rotate( np.array([0.0, 1.0, 0.0]),vec, center=atoms[idx].position)
        atoms.append(addatoms[0])
        atoms.append(addatoms[1])
        atoms.append(addatoms[2])

        atmnums = atoms.get_atomic_numbers()
        tags = atoms.get_tags()

    formula = atoms.get_chemical_formula()

    with output_existing_calculations:
        clear_output()
        # search for existing structures
        qb = QueryBuilder()
        qb.append(StructureData)
        qb.append(CalcJob, filters={'extras.formula':formula}, descendant_of=StructureData)
        qb.order_by({CalcJob:{'ctime':'desc'}})
        for n in qb.iterall():
            calc = n[0]
            print("Found existing calculation: PK=%d | %s"%(calc.pk, calc.get_extra("structure_description")))
            thumbnail = b64decode(calc.get_extra("thumbnail"))
            display(Image(data=thumbnail))

    inp_descr.value = node.description + ".edit" + " " + formula   if node else ""
    update_viewer()

In [ ]:
viewer = nglview.NGLWidget()
viewer.observe(on_picked, names='picked')
tool = ipw.Dropdown(description="On click: ",
                    options=["remove atom",
                             "z UP","z DW",
                             "add H", "add H2", "add H3",
                             "spin 0", "spin up", "spin down",
                             "kind B","kind C","kind N"])

output_existing_calculations = ipw.Output()
display(viewer, tool)

## Step 3: Store structure in the AiiDA database

In [ ]:
# using a workfunction to create link to original structure
@calcfunction
def edit(orig_struct, args):
    atoms = orig_struct.get_ase()
    removals = args.get_array("removals")
    for i in removals: # remove atoms
        del atoms[i]
    for i in addatoms: # add H
        atoms.append(i)
        
    tags = args.get_array("tags")
    atmnums = args.get_array("atmnums")
    positions = args.get_array("positions")
    atoms.set_tags(tags)
    atoms.set_atomic_numbers(atmnums)
    atoms.set_positions(positions)
    atoms.set_masses() # reset to atomic_masses
    new_struct = StructureData(ase=atoms)
    
    # ensure that tags got correctly translated into kinds 
    for t1, k in zip(tags, new_struct.get_site_kindnames()):
        t2 = int(k[-1]) if k[-1].isnumeric() else 0
        assert t1==t2
    return new_struct

In [ ]:
def on_click_store(b):
    global atoms, removals , addatoms ,positions
    
    orig_struct = struct_browser.results.value
    args = ArrayData()
    args.set_array("removals", np.array(removals))
    args.set_array("tags", atoms.get_tags())
    args.set_array("atmnums", atoms.get_atomic_numbers())
    args.set_array("positions", atoms.get_positions())
    print(args)
    s = edit(orig_struct, args)
    s.description = inp_descr.value
    s.store()
    print("Stored in AiiDA: "+repr(s))
    
inp_descr = ipw.Text(placeholder="Description (optional)")   
btn_store = ipw.Button(description='Store in AiiDA')
btn_store.on_click(on_click_store)
display(ipw.HBox([btn_store, inp_descr]))